In [63]:
# coding: utf-8
from typing import Dict, List

import pandas as pd


class Datos:
    datos: int
    nominalAtributos: List[bool]

    # Constructor: procesar el fichero para asignar correctamente las variables nominalAtributos, datos y diccionarios
    def __init__(self, nombreFichero: str):
        self.datosCrudos = pd.read_csv(nombreFichero)
        self.datos = self.datosCrudos.copy()

        self.nominalAtributos = []
        self.diccionarios = {}

        for columna in self.datos.columns:
            if self._es_nominal(columna):
                self.nominalAtributos.append(True)
                self.diccionarios[columna] = self._generar_mapeo(columna)
                self.datos[columna] = self._reemplazar_valores(columna)
            elif self._es_numerico(columna):
                self.nominalAtributos.append(False)
                self.diccionarios[columna] = {}
            else:
                raise ValueError(
                    f"La columna '{columna}' contiene valores que no son nominales ni enteros/decimales."
                )

    def _es_nominal(self, columna: str) -> bool:
        # es verdadero si es la columna objetivo o si sus valores son nominales
        return columna.lower() == "class" or self.datos[columna].dtype.name == "object"

    def _es_numerico(self, columna: str) -> bool:
        # es verdadero si los valores son un n�meros enteros o reales
        return self.datos[columna].dtype.name in ["int64", "float64"]

    def _generar_mapeo(self, columna_nominal: str):
        # se extraen los valores �nicos de la columna y se sortean lexicograficamente
        valores = [str(valor) for valor in self.datos[columna_nominal].unique()]
        valores = sorted(valores)

        return {valor: indice for indice, valor in enumerate(valores)}

    def _reemplazar_valores(self, columna_nominal: str) -> pd.Series:
        mapeo = self.diccionarios[columna_nominal]
        return self.datos[columna_nominal].map(lambda valor: mapeo[str(valor)])

    # Devuelve el subconjunto de los datos cuyos �ndices se pasan como argumento
    def extraeDatos(self, idx: List[int]):
        return self.datos.iloc[idx]


In [64]:
import random
from abc import ABCMeta, abstractmethod
from math import floor
from typing import List

import pandas as pd


class Particion:
    # Esta clase mantiene la lista de indices de Train y Test para cada particion del conjunto de particiones
    def __init__(self, indicesTrain: List[int] = [], indicesTest: List[int] = []):
        self.indicesTrain = indicesTrain
        self.indicesTest = indicesTest

    def __str__(self):
        return f"indices train: {self.indicesTrain}. indices test: {self.indicesTest}"


#####################################################################################################


class EstrategiaParticionado:
    # Clase abstracta
    __metaclass__ = ABCMeta
    particiones: List[Particion]

    def __init__(self):
        self.particiones = []

    # Atributos: deben rellenarse adecuadamente para cada estrategia concreta. Se pasan en el constructor

    @abstractmethod
    # TODO: esta funcion deben ser implementadas en cada estrategia concreta
    def creaParticiones(self, datos: pd.DataFrame, seed: int = None) -> List[Particion]:
        pass


#####################################################################################################


class ValidacionSimple(EstrategiaParticionado):
    def __init__(self, numeroEjecuciones: int, proporcionTest: int):
        super().__init__()
        self.numeroEjecuciones = numeroEjecuciones
        self.proporcionTest = proporcionTest

    # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado y el numero de ejecuciones deseado
    # Devuelve una lista de particiones (clase Particion)
    def creaParticiones(self, datos: pd.DataFrame, seed: int = 42) -> List[Particion]:
        n_filas = datos.shape[0]
        indices = list(range(n_filas))

        random.seed(seed)

        for _ in range(self.numeroEjecuciones):
            random.shuffle(indices)

            # se calcula el numero de ejemplos que se usaran como conjunto de prueba
            proporcion = floor(self.proporcionTest / 100 * n_filas)

            indices_train = indices[proporcion:]
            indices_test = indices[:proporcion]

            particion = Particion(indicesTrain=indices_train, indicesTest=indices_test)

            self.particiones.append(particion)

        return self.particiones


#####################################################################################################
class ValidacionCruzada(EstrategiaParticionado):
    def __init__(self, numeroParticiones: int):
        super().__init__()
        self.numeroParticiones = numeroParticiones

    # Crea particiones segun el metodo de validacion cruzada.
    # El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
    # Esta funcion devuelve una lista de particiones (clase Particion)
    def creaParticiones(self, datos: pd.DataFrame, seed: int = None) -> List[Particion]:
        n_filas = datos.shape[0]
        indices = list(range(n_filas))

        if seed is not None:
            random.seed(seed)
            random.shuffle(indices)

        longitud_fold = n_filas // self.numeroParticiones
        resto = n_filas % self.numeroParticiones

        inicio = 0

        for i in range(self.numeroParticiones):
            fin = inicio + longitud_fold

            # para las longitudes que no son divisibles enteramente por el numero
            # de particiones los primeros folds tienen una longitud extra.
            # e.g. datos = [1,2,3,4,5]; particiones = 3; folds = [[1,2], [3,4], [5]]
            if i < resto:
                fin += 1

            fin = min(fin, n_filas)

            # se construyen los indices para las particiones.
            # Los indices para training son todos aquellos que no pertenecen
            # a los indices de testing
            indices_test = indices[inicio:fin]
            indices_train = [
                indices[j] for j in range(n_filas) if j not in indices_test
            ]

            particion = Particion(indicesTrain=indices_train, indicesTest=indices_test)

            self.particiones.append(particion)

            inicio = fin

        return self.particiones


In [89]:
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import Clasificador

class ClasificadorRegresionLogistica:

    def __init__(self, k_apre):
        self.k_apre = k_apre
    
    

    
    def sigmoid(self, w_v, x_v):
        x = np.array(x_v)[0]
        w = np.array(w_v)[0]
        return 1 / (1 + np.exp(-w*x))


    def actualiza_w(self, w, x):
        t = x.iloc[-1]
        #print(t)
        x = np.array(x.iloc[:-1])
        return w - x * (self.sigmoid(w, x) - t) * self.k_apre
    
    def entrenamiento(self, epocas, datosTrain):
        # crea umbral
        w = [random.uniform(-0.5, 0.5) for _ in datosTrain.iloc[:,:-1]]

        # normalizar los datos
        #for col in datosTrain.columns:
         #   datosTrain[col] = datosTrain[col] / datosTrain[col].abs().max()

        for _ in range(0, epocas):
            for _, dato in datosTrain.iterrows():
                #verosimilitud_clase1 = self.sigmoid(w, dato[:-1])
                w = self.actualiza_w(w, dato)
        return w
    
    def clasifica(self, datosTest, w):
        # normalizar los datos
        #for col in datosTest.columns:
         #   datosTest[col] = datosTest[col] / datosTest[col].abs().max()
        
        precision = 0
        n_datosTest = len(datosTest)



        resultados = pd.DataFrame(columns=["Prediccion", "Clase verdadera"])

        for _, dato in datosTest.iterrows():
            
            X_test = dato.iloc[:-1]
            y_test = dato.iloc[-1]
            prediccion = round(self.sigmoid(w, X_test))

            if prediccion == y_test:
                precision += 1
            
            
            resultados.loc[len(resultados)] = [prediccion, y_test]
        
        print(f"Aciertos: {round(100*precision/n_datosTest, 2)}%")
        return resultados



def resultados_log(k_apre, epocas):
    datos = Datos("wdbc.csv")
    
    # normalizar los datos
    for col in datos.datos.iloc[:, :-1]:
        #datos.datos[col] = datos.datos[col] / datos.datos[col].abs().max()
        datos.datos[col] = (datos.datos[col] - datos.datos[col].mean()) / datos.datos[col].std()
    
    validacion = ValidacionSimple(3, 20)
    particionado = validacion.creaParticiones(datos.datos)
    

    for particion in particionado:
        datos_test = datos.extraeDatos(particion.indicesTest)
        datos_train = datos.extraeDatos(particion.indicesTrain)

    clasificador = ClasificadorRegresionLogistica(k_apre)

    hiperplano = clasificador.entrenamiento(epocas, datos_train)

    res = clasificador.clasifica(datos_test, hiperplano)

    print(res.head(20))
    #print(res.loc[res['Prediccion'] >= 0.543])

resultados_log(4, 10)

"""
class ClasificadorRegresionLogisticaSkLearn:
    atributos = datos.iloc[:, :-1]
    objetivo = datos[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(atributos, objetivo, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    modelo = LogisticRegression()
    modelo.fit(X_train_scaled, y_train)
    predicciones = modelo.predict(X_test_scaled)
"""

Aciertos: 87.61%
    Prediccion  Clase verdadera
0          1.0              1.0
1          0.0              0.0
2          0.0              0.0
3          0.0              0.0
4          0.0              0.0
5          0.0              0.0
6          1.0              1.0
7          0.0              0.0
8          1.0              1.0
9          1.0              1.0
10         0.0              0.0
11         0.0              0.0
12         1.0              1.0
13         1.0              1.0
14         0.0              0.0
15         0.0              0.0
16         1.0              1.0
17         0.0              0.0
18         0.0              0.0
19         1.0              1.0


'\nclass ClasificadorRegresionLogisticaSkLearn:\n    atributos = datos.iloc[:, :-1]\n    objetivo = datos[:, -1]\n\n    X_train, X_test, y_train, y_test = train_test_split(atributos, objetivo, test_size=0.2, random_state=42)\n    scaler = StandardScaler()\n    X_train_scaled = scaler.fit_transform(X_train)\n    X_test_scaled = scaler.transform(X_test)\n\n    modelo = LogisticRegression()\n    modelo.fit(X_train_scaled, y_train)\n    predicciones = modelo.predict(X_test_scaled)\n'